<a href="https://colab.research.google.com/github/masumbillah21/AI-Agents/blob/main/Module4Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > output.log 2>&1 &
!ollama pull phi4

In [ ]:
!pip install ollama faiss-cpu sentence-transformers numpy

Unoptimized Agent

In [ ]:
import ollama
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import time
import logging
from typing import List, Tuple

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class SimpleAgent:
    def __init__(self):
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.dimension = 384  # Dimension of the embedding model
        self.index = faiss.IndexFlatL2(self.dimension)
        self.documents = []
        self.token_usage = 0
        self.query_times = []

    def embed_text(self, text: str) -> np.ndarray:
        """Generate embedding for a given text."""
        return self.embedding_model.encode([text])[0]

    def add_document(self, document: str):
        """Add a document to the FAISS index."""
        start_time = time.time()
        embedding = self.embed_text(document)
        self.index.add(np.array([embedding], dtype=np.float32))
        self.documents.append(document)
        logger.info(f"Added document. Time: {time.time() - start_time:.4f}s")

    def query(self, question: str, k: int = 1) -> Tuple[str, int]:
        """Query the agent with a question."""
        start_time = time.time()

        # Generate embedding for the question
        question_embedding = self.embed_text(question)

        # Search FAISS index
        distances, indices = self.index.search(np.array([question_embedding], dtype=np.float32), k)

        # Retrieve relevant documents
        context = [self.documents[idx] for idx in indices[0] if idx < len(self.documents)]
        context_text = "\n".join(context)

        # Prepare prompt (unoptimized: verbose and redundant)
        prompt = f"""
        You are a helpful assistant. Given the following context and question, provide a detailed answer.

        Context:
        {context_text}

        Question:
        {question}

        Please provide a comprehensive response with all relevant details.
        """

        # Call Ollama Phi-4
        response = ollama.chat(
            model='phi4',
            messages=[{'role': 'user', 'content': prompt}]
        )

        # Track token usage (approximate)
        prompt_tokens = len(prompt.split())
        response_tokens = len(response['message']['content'].split())
        total_tokens = prompt_tokens + response_tokens
        self.token_usage += total_tokens

        # Track query time
        query_time = time.time() - start_time
        self.query_times.append(query_time)

        logger.info(f"Query processed. Time: {query_time:.4f}s, Tokens: {total_tokens}")

        return response['message']['content'], total_tokens

    def get_performance_metrics(self) -> dict:
        """Return performance metrics."""
        return {
            'total_token_usage': self.token_usage,
            'average_query_time': np.mean(self.query_times) if self.query_times else 0,
            'number_of_queries': len(self.query_times)
        }

# Example usage
if __name__ == "__main__":
    agent = SimpleAgent()

    # Add sample documents
    documents = [
        "The capital of France is Paris.",
        "Python is a popular programming language.",
        "The sun is a star."
    ]

    for doc in documents:
        agent.add_document(doc)

    # Test queries
    queries = [
        "What is the capital of France?",
        "What is Python?",
        "Is the sun a star?"
    ]

    for query in queries:
        answer, tokens = agent.query(query)
        print(f"Query: {query}")
        print(f"Answer: {answer}")
        print(f"Tokens used: {tokens}\n")

    # Print performance metrics
    metrics = agent.get_performance_metrics()
    print("Performance Metrics:")
    print(f"Total Token Usage: {metrics['total_token_usage']}")
    print(f"Average Query Time: {metrics['average_query_time']:.4f}s")
    print(f"Number of Queries: {metrics['number_of_queries']}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Query: What is the capital of France?
Answer: The capital of France is Paris. Located in the northern part of the country along the Seine River, Paris is not only the political and administrative center of France but also an influential cultural hub known worldwide for its art, fashion, gastronomy, and culture.

### Historical Context:
Paris has a rich history that dates back to its founding by the Parisii tribe around 3rd century BC. It became one of the most important cities in medieval Europe during the reign of King Philip II in the late 12th century when he transformed it into a thriving metropolis. Throughout the centuries, it has been central to many significant historical events, including the French Revolution and both World Wars.

### Political Significance:
As the capital city, Paris houses the official residences of the President of France (Élysée Palace), the Prime Minister (Hôtel Matignon), and the Parliament of France. It serves as a focal point for national governance a

You Answer: Optimized Agent with performance metrics

Optimization Techniques Implemented:
(List down the techniques you've used)

*   


In [ ]:
#You need to write the agent here in this cell with optimization

Performance Improvement: (Comparision)